# 1. Thư viện

In [1]:
import os
import json

from langchain_community.embeddings import GPT4AllEmbeddings
from dotenv import load_dotenv
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client.models import Filter, FieldCondition, MatchValue

# 2. Mô hình embedding

## 2.1. Mô hình Dense embedding

In [2]:
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

model_Dense_Embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
    )

# 3. Kết nối đến Collection

In [3]:
URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")

In [4]:
exist_Qdrant = QdrantVectorStore.from_existing_collection(
    collection_name = "dense_TinhHuong",
    embedding = model_Dense_Embedding,
    url = URL_QDRANT,
    api_key = API_QDRANT,
    retrieval_mode=RetrievalMode.DENSE
)

# 4. Similarity Search kèm Metadata filtering

In [20]:
user_Query = "Tôi bị mất sổ BHXH, sẽ được cấp lại thế nào?"

query_words = user_Query.lower().split()

# Tạo bộ lọc với danh sách từ khóa trong `query_words`
user_Filter = Filter(
    should=[
        FieldCondition(
            key="metadata.key_word",
            match=MatchValue(value=word)
        ) for word in query_words
    ]
)

print(user_Filter)

should=[FieldCondition(key='metadata.key_word', match=MatchValue(value='tôi'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_word', match=MatchValue(value='bị'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_word', match=MatchValue(value='mất'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_word', match=MatchValue(value='sổ'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_word', match=MatchValue(value='bhxh,'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.key_word', match=MatchValue(value='sẽ'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadat

In [21]:
results = exist_Qdrant.similarity_search_with_score(
    query = user_Query,
    filter = user_Filter,
    k = 5
)

In [23]:
results

[(Document(page_content='Sổ đỏ bị mất, xin cấp lại thế nào?', metadata={'key_word': ['sổ', 'đỏ', 'bị', 'mất', 'xin', 'cấp', 'lại', 'thế', 'nào'], 'context': 'Anh tôi có căn chung cư đã được cấp Giấy chứng nhận quyền sử dụng ngày 1/2/2019. Do bất cẩn trong lưu giữ mà anh tôi đã bị mất sổ đỏ. Tôi có tìm hiểu quy định tại Điều 77 Nghị định số 43/2014/NĐ-CP về thủ tục xin cấp lại sổ đỏ bị mất thì được biết, trước tiên, người bị mất sổ phải đến khai báo với UBND xã/phường nơi có đất. Sau đó UBND xã/phường sẽ niêm yết thông báo công khai tại trụ sở. Sau 30 ngày, người sử dụng đất sẽ nộp 1 bộ hồ sơ xin cấp lại sổ đỏ bị mất. Tuy nhiên, khi anh tôi ra UBND phường khai báo thì lại được yêu cầu sang công an xác minh về việc mất sổ. Khi anh tôi sang công an phường xin xác minh thì phía công an lại từ chối vì cho rằng công an không có thẩm quyền xác minh vấn đề này. Vậy cho tôi hỏi việc anh tôi phải sang công an phường xin xác nhận như vậy có đúng quy định của pháp luật đất đai hay không? Thủ tục x

In [24]:
for doc,score in results:
    print("Question:\n",doc.page_content,"\n")
    print("Context:\n",doc.metadata['context'],"\n")
    print("Score:",score,"\n")

Question:
 Sổ đỏ bị mất, xin cấp lại thế nào? 

Context:
 Anh tôi có căn chung cư đã được cấp Giấy chứng nhận quyền sử dụng ngày 1/2/2019. Do bất cẩn trong lưu giữ mà anh tôi đã bị mất sổ đỏ. Tôi có tìm hiểu quy định tại Điều 77 Nghị định số 43/2014/NĐ-CP về thủ tục xin cấp lại sổ đỏ bị mất thì được biết, trước tiên, người bị mất sổ phải đến khai báo với UBND xã/phường nơi có đất. Sau đó UBND xã/phường sẽ niêm yết thông báo công khai tại trụ sở. Sau 30 ngày, người sử dụng đất sẽ nộp 1 bộ hồ sơ xin cấp lại sổ đỏ bị mất. Tuy nhiên, khi anh tôi ra UBND phường khai báo thì lại được yêu cầu sang công an xác minh về việc mất sổ. Khi anh tôi sang công an phường xin xác minh thì phía công an lại từ chối vì cho rằng công an không có thẩm quyền xác minh vấn đề này. Vậy cho tôi hỏi việc anh tôi phải sang công an phường xin xác nhận như vậy có đúng quy định của pháp luật đất đai hay không? Thủ tục xin cấp lại sổ đỏ bị mất như thế nào?
Bộ Tài nguyên và Môi trường trả lời vấn đề này như sau:Đề nghị 